In [ ]:
from project_utils import  *
import pandas as pd

with open("data/train_data_embedded.json", "r") as f:
    train_data = pd.read_json(f, lines=True)


['index',
 'metric_name',
 'user_prompt',
 'response',
 'system_prompt',
 'score',
 'main_metric']

In [11]:
rel_features = ['metric_name', 'user_prompt', 'response', 'system_prompt']
#create all possible 2 combinations of rel_features
ed_rel_features = []
for i in range(len(rel_features)):
    for j in range(i + 1, len(rel_features)):
        ed_rel_features.append((rel_features[i], rel_features[j]))
ed_only_config = {'euclidean_distances': ed_rel_features,}


fe = FeatureEngineer(train_data, ed_only_config)
df_features = fe.create_features()

TypeError: unsupported operand type(s) for -: 'list' and 'list'